/var/folders/fc/rq9v0gn94dq1cpr9d1jydxq40000gn/T/ipykernel_82908/2480988996.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  nodeset["label"] = nodeset["node"].str.contains(r"(sbox|mixcolumn)",


label
0    195
1     32
Name: count, dtype: int64


In [147]:
feature_names = set(nodeset.columns) - {"node_number", "Node", "node", label}
# feature_names = feature_names - {"and", "or", "xor", "mux"}
# feature_names = feature_names - {"Hamming distance"}
# feature_names = feature_names - {"Paths"}
# feature_names = feature_names - {"Degree"}
feature_names = feature_names - {"Noise"}
num_features = len(feature_names)
num_classes = len(class_idx)

print(train_data[list(feature_names)])
# Create train and test features as a numpy array.
x_train = train_data[list(feature_names)].to_numpy()
x_test = test_data[list(feature_names)].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data[label]
y_test = test_data[label]

     Hamming distance  Degree  Paths  and  or  xor  mux
0                   0       3      1    0   0    0    0
1                   0       2      0    0   0    0    0
2                  60       2     24    0   0    0    0
3                 120       2     24    0   0    0    0
4                   0       5      0    0   0    0    0
..                ...     ...    ...  ...  ..  ...  ...
191                 0       8      0    0   0    0    0
192                 0      24     31    0   0    0    0
193                 8       2     51    0   0    0    0
194                 0       3     93    0   0    0    0
195                 0       3      1    0   0    0    0

[196 rows x 7 columns]


Model: "baseline"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block1          │ (None, 32)        │      1,468 │ input_features[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block2          │ (None, 32)        │      2,368 │ ffn_block1[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection2    │ (None, 32)        │          0 │ ffn_block1[0][0], │
│ (Add)               │                   │            │ ffn_block2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block3          │ (None, 32)        │      2,368 │ skip_connection2… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection3    │ (None, 32)        │          0 │ skip_connection2… │
│ (Add)               │                   │            │ ffn_block3[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block4          │ (None, 32)        │      2,368 │ skip_connection3… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection4    │ (None, 32)        │          0 │ skip_connection3… │
│ (Add)               │                   │            │ ffn_block4[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block5          │ (None, 32)        │      2,368 │ skip_connection4… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection5    │ (None, 32)        │          0 │ skip_connection4… │
│ (Add)               │                   │            │ ffn_block5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ logits (Dense)      │ (None, 1)         │         33 │ skip_connection5… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,973 (42.86 KB)

 Trainable params: 10,383 (40.56 KB)

 Non-trainable params: 590 (2.30 KB)

Epoch 1/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.7192 - loss: 3.6584 - precision_12: 0.0935 - recall_12: 0.0885 - val_acc: 0.9500 - val_loss: 0.8059 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 2/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.6848 - loss: 4.4914 - precision_12: 0.0969 - recall_12: 0.1051 - val_acc: 0.9500 - val_loss: 0.8059 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 3/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.7766 - loss: 3.0414 - precision_12: 0.0927 - recall_12: 0.0814 - val_acc: 0.9500 - val_loss: 0.8059 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 4/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.7474 - loss: 3.4124 - precision_12: 0.1655 - recall_12: 0.1541 - val_acc: 0.9500 - val_loss: 0.8059 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 5/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.6706 - loss: 4.0360 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - val_a

Edges shape: (2, 556)
Nodes shape: (227, 7)


(<tf.Tensor: shape=(227, 7), dtype=float32, numpy=
 array([[  0.,   3.,   1., ...,   0.,   0.,   0.],
        [  0.,   2.,   0., ...,   0.,   0.,   0.],
        [ 60.,   2.,  24., ...,   0.,   0.,   0.],
        ...,
        [456.,   2.,   0., ...,   0.,   0.,   1.],
        [755.,   9.,   1., ...,   0.,   1.,   1.],
        [382.,   6.,   0., ...,   0.,   1.,   1.]], dtype=float32)>,
 array([[ 89, 158, 192, ..., 162, 160,  69],
        [132,  11,  14, ..., 126, 113, 191]]),
 <tf.Tensor: shape=(556,), dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.,

In [152]:


print("GNN output shape:", gnn_model(tf.constant([[1, 10, 100]], dtype=tf.int32)))

gnn_model.summary()

tf.Tensor(
[[[ 0.          0.26289007  0.          0.          0.14481972
    0.          0.12111223  0.33373335  0.          0.
    0.03670914  0.          0.02732704  0.          0.
    0.          0.08023927  0.19005166  0.          0.
    0.          0.43258488  0.          0.07434815  0.25140527
    0.2890036   0.37252447  0.18037213  0.10688798  0.28320923
    0.          0.03093883]
  [ 0.08801861  0.3787389   0.          0.          0.05389495
    0.          0.2517246   0.531079    0.          0.
    0.1027099   0.          0.          0.          0.
    0.          0.07570715  0.14243221  0.          0.
    0.          0.59629625  0.          0.21232678  0.43357742
    0.49898833  0.5355927   0.20774882  0.04955582  0.34113953
    0.          0.        ]
  [ 0.         14.548032    0.          0.52295953  0.
    5.081062    0.          1.8988961   0.          0.
    0.          5.4260917  16.994213    1.1258637   0.25360134
    1.5599507  11.198236    0.          5.7361765   

Model: "gnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocess (Sequential)         │ (227, 32)              │         1,468 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv1 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv2 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ postprocess (Sequential)        │ (227, 32)              │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (1, 3, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,678 (61.24 KB)

 Trainable params: 14,832 (57.94 KB)

 Non-trainable params: 846 (3.30 KB)

Epoch 1/32
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
1/9 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - acc: 1.0000 - loss: 0.0719 - precision_22: 1.0000 - recall_22: 1.0000Tensor("gnn_model_1/GatherV2:0", shape=(20, 32), dtype=float32)
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - acc: 1.0000 - loss: 0.1248 - precision_22: 1.0000 - recall_22: 1.0000 - val_acc: 1.0000 - val_loss: 0.0826 - val_precision_22: 1.0000 - val_recall_22: 1.0000
Epoch 2/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 1.0000 - loss: 0.1310 - precision_22: 1.0000 - recall_22: 1.0000 - val_acc: 1.0000 - val_loss: 0.0778 - val_precision_22: 1.0000 - val_recall_22: 1.0000
Epoch 3/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 1.0000 - loss: 0.1198 - precision_22: 1.0000 - recall_22: 1.0000 - val_acc: 1.0000 - val_loss: 0.0734 - val_precision_22: 1.0000 - val_recall_22: 1.0000
Epoch 4/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 1.0000 - loss: 0

Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Test accuracy: 93.55%
Test precision: 93.54838728904724%
Test recall: 93.54838728904724%


In [156]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred= gnn_model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_testt = np.argmax(y_test1, axis=1)
cm = confusion_matrix(y_testt, y_pred)
print(cm)
print(classification_report(y_testt,y_pred))

Tensor("gnn_model_1/GatherV2:0", shape=(31, 32), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
[[27  0]
 [ 2  2]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        27
           1       1.00      0.50      0.67         4

    accuracy                           0.94        31
   macro avg       0.97      0.75      0.82        31
weighted avg       0.94      0.94      0.93        31



In [157]:
!python Feature_Extract.py AES_PPRM1 Kin AES_PPRM1
!python Feature_Extract.py AES_PPRM3 Kin AES_PPRM3
!python Feature_Extract.py AES_TBL Kin AES_TBL
!python Feature_Extract.py RSA Kin RSA
!python Feature_Extract.py SABER pol_64bit_in SABER

Generating LALR tables
[INFO] Features written to ../test/AES_PPRM1_features.csv
[INFO] Edges written to ../test/AES_PPRM1_edges.csv
Generating LALR tables
[INFO] Features written to ../test/AES_PPRM3_features.csv
[INFO] Edges written to ../test/AES_PPRM3_edges.csv
Generating LALR tables
[INFO] Features written to ../test/AES_TBL_features.csv
[INFO] Edges written to ../test/AES_TBL_edges.csv
Generating LALR tables
[INFO] Features written to ../test/RSA_features.csv
[INFO] Edges written to ../test/RSA_edges.csv
Generating LALR tables
[INFO] Features written to ../test/SABER_features.csv
[INFO] Edges written to ../test/SABER_edges.csv


In [163]:
import os, glob
import numpy as np
import pandas as pd
import tensorflow as tf

TEST_DIR = "../test"

feature_files = sorted(glob.glob(os.path.join(TEST_DIR, "*_features.csv")))
paired = []
for fpath in feature_files:
    base = os.path.basename(fpath).replace("_features.csv", "")
    epath = os.path.join(TEST_DIR, f"{base}_edges.csv")
    if os.path.exists(epath):
        paired.append((base, fpath, epath))

if not paired:
    print("No dataset pairs found under ./test (expect *_features.csv + *_edges.csv)")
else:
    print(f"Found {len(paired)} dataset(s):", [b for b,_,_ in paired])

leaky_module = {
    "AES_PPRM1": ["SBOX", "Mixcolumns", "MX"],
    "AES_PPRM3": ["Sbox", "Mixcolumns", "MX"],
    "AES_TBL": ["SBOX", "Mixcolumns", "MX"],
    "RSA": ["MODEXP_SEQ", "MULT_BLK"],
    "SABER": ["PMULTs"]
}

for base, ffeat, fedge in paired:
    print(f"\n=== [{base}] ===")
    test_nodeset = pd.read_csv(ffeat)
    test_edge = pd.read_csv(fedge)
    keywords = leaky_module.get(base, [])
    def contains_any(value, keywords):
        return any(kw in str(value) for kw in keywords)
    test_nodeset["label"] = test_nodeset["node"].apply(
        lambda x: 1 if contains_any(x, keywords) else 0
    )
    test_nodeset = test_nodeset[list(feature_names)]
    test_nodeset.to_csv(ffeat, index=False)

    test_feature = test_nodeset.node_number.to_numpy()
    test_target = tf.keras.utils.to_categorical(test_nodeset["label"], num_classes=2)

        # ===== Chunked evaluation (avoid indices out of training bound) =====
    # 1) Automatically detect training upper bound (max index = TRAIN_N - 1)
    def _detect_train_N(model):
        # Try to get from Embedding layer
        for layer in model.layers:
            try:
                from tensorflow.keras.layers import Embedding
            except Exception:
                Embedding = tf.keras.layers.Embedding
            if isinstance(layer, Embedding):
                if getattr(layer, "embeddings", None) is not None and layer.embeddings.shape.rank == 2:
                    return int(layer.embeddings.shape[0])
                if getattr(layer, "input_dim", None) is not None:
                    return int(layer.input_dim)
        # Try common variable names (if custom feature table exists)
        for attr in ["feature_table", "node_features", "features"]:
            if hasattr(model, attr):
                var = getattr(model, attr)
                try:
                    return int(var.shape[0])
                except Exception:
                    pass
        return None

    TRAIN_N = _detect_train_N(gnn_model)
    if TRAIN_N is None:
        TRAIN_N = 227  # fallback
        print(f"[{base}] Warning: cannot detect train_N from model; fallback TRAIN_N={TRAIN_N}")

    # 2) Split test set into chunks of at most TRAIN_N samples
    CHUNK = TRAIN_N
    n_total = test_feature.shape[0]

    # Ensure dtype and shape
    test_feature = np.asarray(test_feature, dtype=np.int32)
    test_target  = np.asarray(test_target,  dtype=np.float32)

    total_samples = 0
    acc_sum = 0.0
    prec_sum = 0.0
    rec_sum = 0.0

    for start in range(0, n_total, CHUNK):
        end = min(start + CHUNK, n_total)
        idx_chunk = test_feature[start:end]
        y_chunk   = test_target[start:end]

        unique_idx = np.unique(idx_chunk)
        remap = {old: new for new, old in enumerate(unique_idx)}
        idx_valid = np.array([remap[i] for i in idx_chunk], dtype=np.int32)
        y_valid   = y_chunk

        # Evaluate this chunk
        loss, acc, prec, rec = gnn_model.evaluate(x=idx_valid, y=y_valid, verbose=0)

        m = idx_valid.shape[0]
        total_samples += m
        acc_sum  += float(acc)  * m
        prec_sum += float(prec) * m
        rec_sum  += float(rec)  * m

    if total_samples == 0:
        print(f"[{base}] Warning: no valid indices (< {TRAIN_N}) in test set; skipped.")
    else:
        test_accuracy = acc_sum / total_samples
        precision     = prec_sum / total_samples
        recall        = rec_sum / total_samples
        print(f"Test accuracy: {acc_sum} / {total_samples} = {round(test_accuracy * 100, 2)}%")
        # Optional: print precision/recall as well
        # print(f"Test precision: {round(precision * 100, 2)}%")
        # print(f"Test recall:    {round(recall * 100, 2)}%")


Found 5 dataset(s): ['AES_PPRM1', 'AES_PPRM3', 'AES_TBL', 'RSA', 'SABER']

=== [AES_PPRM1] ===


AttributeError: 'DataFrame' object has no attribute 'node_number'